https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

In [1]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
print("Tensorflow ",tf.__version__)
print("Keras ",keras.__version__)

Tensorflow  1.8.0
Keras  2.2.0


Using TensorFlow backend.


In [15]:
batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_cnn.h5'

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print (y_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)


In [16]:
print(y_train[:3])

[[6]
 [9]
 [9]]


In [17]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[:3])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [18]:
model = Sequential()
model.add(Conv2D(32,3, padding = 'same', input_shape = x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

ops = keras.optimizers.rmsprop(lr = 0.0001, decay  = 1e-6)

model.compile(loss = 'categorical_crossentropy', 
             optimizer = ops,
             metrics = ['accuracy'])

In [19]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last"
        validation_split=0.0)  # fraction of images reserved for validation (strictly between 0 an)

datagen.fit(x_train)

In [20]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                   epochs = epochs,
                   validation_data = (x_test, y_test),
                   workers = 4)

Epoch 1/22
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8514 - acc: 0.3158 - val_loss: 1.6194 - val_acc: 0.4152
Epoch 2/22
1563/1563 [==============================] - 26s 17ms/step - loss: 1.5885 - acc: 0.4199 - val_loss: 1.4507 - val_acc: 0.4726
Epoch 3/22
1563/1563 [==============================] - 27s 17ms/step - loss: 1.4748 - acc: 0.4637 - val_loss: 1.3920 - val_acc: 0.4967
Epoch 4/22
1563/1563 [==============================] - 28s 18ms/step - loss: 1.3927 - acc: 0.4981 - val_loss: 1.2129 - val_acc: 0.5681
Epoch 5/22
1563/1563 [==============================] - 28s 18ms/step - loss: 1.3261 - acc: 0.5217 - val_loss: 1.1837 - val_acc: 0.5761
Epoch 6/22
1563/1563 [==============================] - 28s 18ms/step - loss: 1.2664 - acc: 0.5496 - val_loss: 1.1647 - val_acc: 0.5855
Epoch 7/22
1563/1563 [==============================] - 28s 18ms/step - loss: 1.2179 - acc: 0.5636 - val_loss: 1.0614 - val_acc: 0.6238
Epoch 8/22
1563/1563 [==========================

In [21]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\Harrison\Documents\GitHub\ML-Notes\3. Deep Learning Notes\Keras Exercises\Keras Examples\saved_models\keras_cifar10_cnn.h5 


In [22]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 149us/step
Test loss: 0.7732314324378967
Test accuracy: 0.7307
